In [ ]:
import orchid

In [ ]:
# Remember that importing `orchid` results in a call to `pythonnet.load()`
# to specify the targeted runtime:
#
# 'netfx': .NET Framework
# 'coreclr': .NET Core
# 'mono': Mono

import clr

In [ ]:
from System import DateTime, DateTimeKind

In [ ]:
DateTime

In [ ]:
DateTime.Overloads, DateTime.__overloads__

In [ ]:
type(DateTime.__overloads__)

In [ ]:
dir(DateTime)

In [ ]:
import pythonnet

In [ ]:
pythonnet.get_runtime_info()

In [ ]:
type(DateTimeKind.Utc)

In [ ]:
dir(DateTimeKind.Utc)

In [ ]:
DateTimeKind.Utc.GetType()

In [ ]:
dir(DateTimeKind.Utc.GetType())

In [ ]:
DateTimeKind.Utc.GetType().BaseType

In [ ]:
DateTimeKind.Utc.GetType().BaseType.FullName